# Get backend information with Qiskit

<LegacyContent>
<Admonition type="note">
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.]( https://quantum.cloud.ibm.com/docs/guides/get-qpu-information)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/get-qpu-information)
</Admonition>
</CloudContent>

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit-ibm-runtime~=0.37.0
```
</details>

This page explains how to use Qiskit to find information about your available backends.

## List backends
<LegacyContent>
To view the backends you have access to, you can either view a list on the [Compute resources page,](https://quantum.ibm.com/services/resources?tab=yours) or you can use the [`QiskitRuntimeService.backends()`](../api/qiskit-ibm-runtime/qiskit-runtime-service#backends) method. This method returns a list of [`IBMBackend`](../api/qiskit-ibm-runtime/ibm-backend) instances:
</LegacyContent>
<CloudContent>
To view the backends you have access to, you can either view a list on the [Compute resources page,](https://quantum.cloud.ibm.com/computers) or you can use the [`QiskitRuntimeService.backends()`](../api/qiskit-ibm-runtime/qiskit-runtime-service#backends) method. This method returns a list of [`IBMBackend`](../api/qiskit-ibm-runtime/ibm-backend) instances:
</CloudContent>

<LegacyContent>
To run the following code, be sure you have already authenticated to the service. See [Set up to use IBM Quantum Platform](/docs/guides/setup-channel#set-up-to-use-ibm-quantum-platform) for more details.
</LegacyContent>
<CloudContent>
To run the following code, be sure you have already authenticated to the service. See [Set up your IBM Cloud account](https://quantum.cloud.ibm.com/docs/en/guides/cloud-setup) for more details.
</CloudContent>

In [2]:
# Initialize your account
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")

service.backends()

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_torino')>]

The [`QiskitRuntimeService.backend()`](../api/qiskit-ibm-runtime/qiskit-runtime-service#backend) method (note that this is singular: *backend*) takes the name of the backend as the input parameter and returns an [`IBMBackend`](../api/qiskit-ibm-runtime/ibm-backend) instance representing that particular backend:

In [2]:
service.backend("ibm_brisbane")

<IBMBackend('ibm_brisbane')>

## Filter backends

You can also filter the available backends by their properties. For more general filters, set the `filters` argument to a function that accepts a backend object and returns `True` if it meets your criteria. Refer to the [API documentation](../api/qiskit-ibm-runtime/qiskit-runtime-service#backends) for more details.

The following code returns only backends that fit these criteria:

*   Are real quantum devices (`simulator=False`)
*   Are currently operational (`operational=True`)
*   Have at least 5 qubits (`min_num_qubits=5`)

In [3]:
service.backends(simulator=False, operational=True, min_num_qubits=5)

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_torino')>]

Use these keyword arguments to filter by any attribute in backend configuration ([JSON schema](https://github.com/Qiskit/ibm-quantum-schemas/blob/main/schemas/backend_configuration_schema.json)) or status ([JSON schema](https://github.com/Qiskit/ibm-quantum-schemas/blob/main/schemas/backend_status_schema.json)). A similar method is [`QiskitRuntimeService.least_busy()`](../api/qiskit-ibm-runtime/qiskit-runtime-service#least_busy), which takes the same filters as `backends()` but returns the backend that matches the filters and has the least number of jobs pending in the queue:

In [3]:
service.least_busy(operational=True, min_num_qubits=5)

<IBMBackend('ibm_brisbane')>

## Static backend information

Some information about a backend does not change regularly, such as its name, version, the number of qubits it has, and the types of features it supports. This information is available as attributes of the `backend` object.

The following cell builds a description of a backend.

In [7]:
backend = service.backend("ibm_brisbane")

print(
    f"Name: {backend.name}\n"
    f"Version: {backend.version}\n"
    f"No. of qubits: {backend.num_qubits}\n"
)

Name: ibm_brisbane
Version: 2
No. of qubits: 127



For a full list of attributes, see the [`IBMBackend` API documentation](/docs/api/qiskit-ibm-runtime/ibm-backend).

## Dynamic backend information

Backends can also have properties that change whenever the backed is calibrated, such as qubit frequency and operation error rates. Backends are usually calibrated every 24 hours, and their properties update after the calibration sequence completes. These properties can be used when optimizing quantum circuits or to construct noise models for a classical simulator.


### Qubit properties


The `backend.properties().qubit_property()` returns information about the qubits' physical attributes. It contains a dictionary of various properties of the qubit, each paired with its value and the timestamp of the last calibration.

- `T1 (Relaxation Time)`: The T1 time represents the average duration a qubit remains in its excited state $|1\rangle$ before decaying to its ground state $|0\rangle$ due to energy relaxation. This parameter is used to characterize the qubit's energy relaxation behavior, and is expressed in units of seconds (s).

- `T2 (Dephasing Time)`: The T2 time denotes the timescale over which a qubit maintains phase coherence of a superposition between the $|0\rangle$ and $|1\rangle$ states. It accounts for both energy relaxation and pure dephasing processes, providing insight into the qubit's coherence properties.

- `frequency`: This parameter specifies the resonant frequency of the qubit, indicating the energy difference between the $|0\rangle$ and $|1\rangle$ states, expressed in hertz (Hz).

- `anharmonicity`: Anharmonicity is the difference in energy between the first and second excited states of the qubit, also expressed in hertz (Hz).

- `readout_error`: The readout error quantifies the average probability of incorrectly measuring a qubit's state. It is commonly calculated as the mean of prob_meas0_prep1 and prob_meas1_prep0, providing a single metric for measurement fidelity.

- `prob_meas0_prep1`: This parameter indicates the probability of measuring a qubit in the 0 state when it was intended to be prepared in the $|1\rangle$ state, denoted as $P(0 | 1)$. It reflects errors in state preparation and measurement (SPAM), particularly measurement errors in superconducting qubits.

- `prob_meas1_prep0`: Similarly, this parameter represents the probability of measuring a qubit in the 1 state when it was intended to be prepared in the $|0\rangle$ state, denoted as $P(1 | 0)$. Like prob_meas0_prep1, it reflects SPAM errors, with measurement errors being the predominant contributor in superconducting qubits.

- `readout_length`: The readout_length specifies the duration of the readout operation for a qubit. It measures the time from the initiation of the measurement pulse to the completion of signal digitization, after which the system is ready for the next operation. Understanding this parameter is crucial for optimizing circuit execution, especially when incorporating mid-circuit measurements.

In [6]:
# fundamental physical properties of qubit 1
backend.qubit_properties(1)

QubitProperties(t1=0.000373804819308844, t2=0.00035501732926633675, frequency=4736289915.805671)

In [7]:
# calibration data with detailed properties of qubit 0
backend.properties().qubit_property(0)

{'T1': (0.0005462183129681861,
  datetime.datetime(2025, 6, 4, 18, 41, 14, tzinfo=tzlocal())),
 'T2': (0.0003767959201654903,
  datetime.datetime(2025, 6, 4, 18, 41, 14, tzinfo=tzlocal())),
 'frequency': (4635659902.192823,
  datetime.datetime(2025, 6, 4, 19, 20, 41, tzinfo=tzlocal())),
 'anharmonicity': (-313276039.4092362,
  datetime.datetime(2025, 6, 4, 19, 20, 41, tzinfo=tzlocal())),
 'readout_error': (0.005615234375,
  datetime.datetime(2025, 6, 4, 19, 20, 41, tzinfo=tzlocal())),
 'prob_meas0_prep1': (0.0068359375,
  datetime.datetime(2025, 6, 4, 19, 20, 41, tzinfo=tzlocal())),
 'prob_meas1_prep0': (0.00439453125,
  datetime.datetime(2025, 6, 4, 19, 20, 41, tzinfo=tzlocal())),
 'readout_length': (1.216e-06,
  datetime.datetime(2025, 6, 4, 19, 20, 41, tzinfo=tzlocal()))}

In [8]:
# Retrieve qubit properties
qubit_index = 126  # Replace with your qubit index
qubit_props = backend.properties().qubit_property(qubit_index)

# Access specific properties
t1 = qubit_props.get("T1", (None,))[0]
t2 = qubit_props.get("T2", (None,))[0]
frequency = qubit_props.get("frequency", (None,))[0]
anharmonicity = qubit_props.get("anharmonicity", (None,))[0]
readout_error = qubit_props.get("readout_error", (None,))[0]
prob_meas0_prep1 = qubit_props.get("prob_meas0_prep1", (None,))[0]
prob_meas1_prep0 = qubit_props.get("prob_meas1_prep0", (None,))[0]
readout_length = qubit_props.get("readout_length", (None,))[0]

print(f"Qubit {qubit_index} Properties:")
print(f"  T1: {t1} seconds")
print(f"  T2: {t2} seconds")
print(f"  Frequency: {frequency} Hz")
print(f"  Anharmonicity: {anharmonicity} Hz")
print(f"  Readout Error: {readout_error}")
print(f"  P(0 | 1): {prob_meas0_prep1}")
print(f"  P(1 | 0): {prob_meas1_prep0}")
print(f"  Readout Length: {readout_length} seconds")

Qubit 126 Properties:
  T1: 0.0003018540542473072 seconds
  T2: 9.515509303172122e-05 seconds
  Frequency: 4831245145.564327 Hz
  Anharmonicity: -309300543.2479466 Hz
  Readout Error: 0.01416015625
  P(0 | 1): 0.0166015625
  P(1 | 0): 0.01171875
  Readout Length: 1.216e-06 seconds


### Instruction properties

The `backend.target` attribute is a `qiskit.transpiler.Target` object: an object that contains all the information needed to transpile a circuit for that backend. This includes instruction errors and durations. For example, the following cell gets the properties for an [`ecr` gate](/docs/api/qiskit/qiskit.circuit.library.ECRGate) acting between qubits 1 and 0.

In [7]:
backend.target["ecr"][(1, 0)]

InstructionProperties(duration=5.333333333333332e-07, error=0.006190706171161048)

The following cell shows the properties for a measurement operation (including the readout error) on qubit 0.

In [8]:
backend.target["measure"][(0,)]

InstructionProperties(duration=1.216e-06, error=0.005615234375)

## Next steps

<Admonition type="tip" title="Recommendations">
    - Try the [Grover's algorithm](https://learning.quantum.ibm.com/tutorial/grovers-algorithm) tutorial.
    - Review the [QiskitRuntime backend API](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#backend) reference.
</Admonition>